# Build Interactive Dashboard for Launch Data


This notebook builds a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in
real-time.

**Contents:**  
- Drop-down for user input for Launch Site
- Pie-chart based on dropdown selection
- Range Slider for user input for Payload Mass
- Scatterplot based on Payload selection

#### Install packages; import libraries

In [1]:
!pip3 install pandas dash
!pip3 install wget

In [3]:
import wget
import pandas as pd

import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
from dash import callback
import plotly.express as px

#### **Fetch User Input Parameters**
- Extract list of launch sites for dropdown menu
- Get range of payload mass for slider 

In [6]:
# SpaceX Launch dataset
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')
spacex_df=pd.read_csv(spacex_csv_file)
spacex_df.shape
#

(56, 7)

In [8]:
# get list of launch sites

launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
sites = list(launch_sites_df['Launch Site'])
allsites =['All Sites']
sites_opt = allsites + sites
type(sites_opt)

list

In [10]:
# Get range of payload mass

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

### **Creating Dashboard**

In [17]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Div([
                                        # Create an division for adding dropdown helper text for report type
                                        html.Div(
                                                  [
                                                  html.H2('Launch Site:', style={'margin-right': '2em'}),
                                                  ]
                                                ),
                                        # Add a dropdown
                                        dcc.Dropdown(id='site-dropdown', 
                                                     options = sites_opt, 
                                                     placeholder='All Sites',
                                                     style={'textAlign': 'center', 'color': '#000000', 'font-size': 20, 'width':'80%', 'padding':'3px'}
                                                     )
                                            ], style={'display':'flex'}),

                                html.Br(),

                                # Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000, 
                                                marks={0: '0', 2500: '2500', 5000: '5000', 
                                                       7500: '7500', 10000: '10000'}, value=[min_payload, max_payload]),
                                
                                # Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])


# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    d = spacex_df
    if entered_site == 'All Sites':
        fig = px.pie(d, values='class', names='Launch Site', title='Total Number of Successful Launches for All Sites')
        fig.update_traces(textinfo='value') # shows counts instead of percentages
        return fig
    else:
        # return the outcomes piechart for a selected site
        data = d[d['Launch Site'] == entered_site]
        ttl = 'Launch Outcome for Selected Site'
        fig = px.pie(data, 
                     #values= 'class', 
                     names='class', 
                     # labels={0: "Gender", 1: "Smokes"}, # cannot make this work
                     title= ttl)
        # fig.update_traces(textinfo='value')
        return fig

    

# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output    
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])


def get_scatterchart(entered_site, sldr_payload):
    d = spacex_df
    if entered_site == 'All Sites':
        drange = d[(d['Payload Mass (kg)']>=sldr_payload[0])
        &(d['Payload Mass (kg)']<=sldr_payload[1])]
        fig = px.scatter(drange, x="Payload Mass (kg)", y="class", color="Booster Version Category")
        return fig
        # If ALL sites are selected, render a scatter plot to display all values for variable Payload Mass (kg) 
        # and variable class. In addition, the point color needs to be set to the booster version 
        # i.e., color="Booster Version Category"
    else:
        data = d[(d['Launch Site'] == entered_site)]
        filtered_data = data[(data['Payload Mass (kg)']>=sldr_payload[0])
        &(data['Payload Mass (kg)']<=sldr_payload[1])]
            #sx -- filter data with payload slider!
            #sx -- sldr_payload is a list with two components: min and max
        ttl = 'ttitle'
        fig = px.scatter(filtered_data, x="Payload Mass (kg)", y="class", color="Booster Version Category")
        return fig
        # If a specific launch site is selected, you need to filter the spacex_df first, 
        # and render a scatter chart to show values Payload Mass (kg) and class for the selected site, 
        # and color-label the point using Boosster Version Category likewise.
        
    
# Run the app
if __name__ == '__main__':
    app.run_server(port=2223)

#### **Insights from Dashboard**

From piechart:   
- Site KSC LC-39A has the greatest number of successes at 41.7%.
- Site CCAFS SLC-40 has the lowest number of success at 12.5% of total number of successful outcomes.
- Site KSC LC-39A has highest ratio of successes at 76.9%.


From payload scatterplot:    
- Launches with payload between 3000 and 5000 have the highest rate of success. 
- Payload higher than 5500 is associated with the lowest likelihood of success.
- Launches with booster version FT is the most likely to succeed (2 out of 3 launches are successful).
- Booster versions v1.0 are v1.1 rarely had any success. While success rate for B4 version launches are about 50%.

#### Plotly Dash Reference

Dropdown (input) component
Refer [here](https://dash.plotly.com/dash-core-components/dropdown?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01) for more details about dcc.Dropdown()

Range slider (input) component
Refer [here](https://dash.plotly.com/dash-core-components/rangeslider?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01)] for more details about dcc.RangeSlider()

Pie chart (output) component
Refer [here](https://plotly.com/python/pie-charts/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01) for more details about plotly pie charts

Scatter chart (output) component
Refer [here](https://plotly.com/python/line-and-scatter/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01) for more details about plotly scatter charts